In [ ]:
import pandas as pd
import sys
import numpy as np
from numpy import cos, sin, arcsin, sqrt
from datetime import datetime
%pip install geopandas
import geopandas as gpd
%pip install folium
import folium
from folium.plugins import MarkerCluster
import branca.colormap
%pip install mplleaflet
import mplleaflet
import warnings
from pyproj import Proj, transform
warnings.filterwarnings("ignore")
import json
from IPython.display import display

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 8.7 MB/s 
     |████████████████████████████████| 16.7 MB 29.1 MB/s 
     |████████████████████████████████| 6.3 MB 44.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for mplleaflet: filename=mplleaflet-0.0.5-py3-none-any.whl size=28582 sha256=1e2bae5fbbb41ea7f5ad1334165c7d9db8a55f0d35cfc9c2978f4b32c423be89
  Stored in directory: /root/.cache/pip/wheels/6b/f5/21/cdd12e476182b4b0b98326cdb9efa02ddbd5d87ca5de051c84
Successfully built mplleaflet


In [ ]:
data_file_path = "/content/peachtree_acceleration.csv"

In [ ]:
header_list=["vehicle_ID", "Global_Time", "Global_X", "Global_Y", "v_Class", "v_Vel", "v_Acc", "Lane_ID", "Location"]

In [ ]:
df = pd.read_csv(data_file_path, skiprows=355, nrows=196) # trip_ID=11 #nrows=356,172

In [ ]:
df.columns=header_list

In [ ]:
df

,vehicle_ID,Global_Time,Global_X,Global_Y,v_Class,v_Vel,v_Acc,Lane_ID,Location
0,11,1163019300,2230835.218,1377482.996,2,14.99,0.00,0,peachtree
1,11,1163021100,2230811.322,1377471.406,2,17.61,-0.77,0,peachtree
2,11,1163021700,2230804.169,1377464.210,2,16.16,-0.98,0,peachtree
3,11,1163022100,2230800.104,1377458.985,2,21.16,10.02,0,peachtree
4,11,1163023500,2230794.597,1377423.065,2,29.64,9.64,2,peachtree
...,...,...,...,...,...,...,...,...,...
191,11,1163147800,2230508.838,1375763.434,2,9.05,0.00,0,peachtree
192,11,1163149200,2230507.077,1375750.901,2,9.04,0.00,0,peachtree
193,11,1163150900,2230504.939,1375735.685,2,9.04,0.00,0,peachtree
194,11,1163152600,2230501.533,1375705.138,2,28.26,0.00,0,peachtree


In [ ]:
def get_clean_time_stamp(epoch, req):
    if req == 'day_name':
        return datetime.fromtimestamp(epoch).strftime("%b")
    elif req == 'month_name':
        return datetime.fromtimestamp(epoch).strftime("%a")
    elif req == 'day_num':
        return datetime.fromtimestamp(epoch).strftime("%d")
    elif req == 'month_num':
        return datetime.fromtimestamp(epoch).strftime("%m")
    elif req == 'year_num':
        return datetime.fromtimestamp(epoch).strftime("%Y")
    elif req == 'week_num':
        return datetime.fromtimestamp(epoch).strftime("%U")
    elif req == 'weekday_num':
        return datetime.fromtimestamp(epoch).strftime("%w")
    elif req == 'hour':
        return datetime.fromtimestamp(epoch).strftime("%H")
    elif req == 'min':
        return datetime.fromtimestamp(epoch).strftime("%M")
    elif req == 'sec':
        return datetime.fromtimestamp(epoch).strftime("%S")
    elif req == 'name_combo':
        return f"{datetime.fromtimestamp(epoch).strftime('%a')},{datetime.fromtimestamp(epoch).strftime('%b')}"
    elif req == 'num_combo':
        return f"{datetime.fromtimestamp(epoch).strftime('%m')},{datetime.fromtimestamp(epoch).strftime('%d')},{datetime.fromtimestamp(epoch).strftime('%Y')}"
    elif req == 'time_combo':
        # return f"{datetime.fromtimestamp(epoch).strftime('%H')}:{datetime.fromtimestamp(epoch).strftime('%M')}"
        return f"{datetime.fromtimestamp(epoch).strftime('%H')},{datetime.fromtimestamp(epoch).strftime('%M')},{datetime.fromtimestamp(epoch).strftime('%S')}"

In [ ]:
df['day_num'] = df['Global_Time'].astype(int).apply(get_clean_time_stamp, args=['day_num'])
df['month_num'] = df['Global_Time'].astype(int).apply(get_clean_time_stamp, args=['month_num'])
df['year_num'] = df['Global_Time'].astype(int).apply(get_clean_time_stamp, args=['year_num'])
df['week_num'] = df['Global_Time'].astype(int).apply(get_clean_time_stamp, args=['week_num'])
df['weekday_num'] = df['Global_Time'].astype(int).apply(get_clean_time_stamp, args=['weekday_num'])
df['hours'] = df['Global_Time'].astype(int).apply(get_clean_time_stamp, args=['hour'])
df['minutes'] = df['Global_Time'].astype(int).apply(get_clean_time_stamp, args=['min'])
df['seconds'] = df['Global_Time'].astype(int).apply(get_clean_time_stamp, args=['sec'])

In [ ]:
inProj = Proj('epsg:2240') # Georgia State Plane Coordinate System, Georgia West
outProj = Proj('epsg:4326')
df['Latitude'], df['Longitude'] = transform(inProj, outProj, df['Global_X'].tolist(), df['Global_Y'].tolist())

In [ ]:
df.head()

,vehicle_ID,Global_Time,Global_X,Global_Y,v_Class,v_Vel,v_Acc,Lane_ID,Location,day_num,month_num,year_num,week_num,weekday_num,hours,minutes,seconds,Latitude,Longitude
0,11,1163019300,2230835.218,1377482.996,2,14.99,0.00,0,peachtree,08,11,2006,45,3,20,55,00,33.786609,-84.383068
1,11,1163021100,2230811.322,1377471.406,2,17.61,-0.77,0,peachtree,08,11,2006,45,3,21,25,00,33.786577,-84.383147
2,11,1163021700,2230804.169,1377464.210,2,16.16,-0.98,0,peachtree,08,11,2006,45,3,21,35,00,33.786557,-84.383170
3,11,1163022100,2230800.104,1377458.985,2,21.16,10.02,0,peachtree,08,11,2006,45,3,21,41,40,33.786543,-84.383183
4,11,1163023500,2230794.597,1377423.065,2,29.64,9.64,2,peachtree,08,11,2006,45,3,22,05,00,33.786444,-84.383201


In [ ]:
df.to_excel("output.xlsx")

In [ ]:
#df.Global_Time = pd.to_datetime(df.Global_Time, format='%Y-%m-%d %H:%M:%S', errors='ignore')

In [ ]:
m = folium.Map(location = [33.784, -84.3835], zoom_start = 14, max_zoom = 19)
feature_group0 = folium.FeatureGroup(name = "velocity")
feature_group1 = folium.FeatureGroup(name = "acceleration")
feature_group2 = folium.FeatureGroup(name = "marker")
feature_group3 = folium.FeatureGroup(name = "speeding/hard brake")
marker_cluster = MarkerCluster().add_to(feature_group2)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(x = df.Latitude, y = df.Longitude), crs = "EPSG:3857")

In [ ]:
gdf.head()

,vehicle_ID,Global_Time,Global_X,Global_Y,v_Class,v_Vel,v_Acc,Lane_ID,Location,day_num,month_num,year_num,week_num,weekday_num,hours,minutes,seconds,Latitude,Longitude,geometry
0,11,1163019300,2230835.218,1377482.996,2,14.99,0.00,0,peachtree,08,11,2006,45,3,20,55,00,33.786609,-84.383068,POINT (33.787 -84.383)
1,11,1163021100,2230811.322,1377471.406,2,17.61,-0.77,0,peachtree,08,11,2006,45,3,21,25,00,33.786577,-84.383147,POINT (33.787 -84.383)
2,11,1163021700,2230804.169,1377464.210,2,16.16,-0.98,0,peachtree,08,11,2006,45,3,21,35,00,33.786557,-84.383170,POINT (33.787 -84.383)
3,11,1163022100,2230800.104,1377458.985,2,21.16,10.02,0,peachtree,08,11,2006,45,3,21,41,40,33.786543,-84.383183,POINT (33.787 -84.383)
4,11,1163023500,2230794.597,1377423.065,2,29.64,9.64,2,peachtree,08,11,2006,45,3,22,05,00,33.786444,-84.383201,POINT (33.786 -84.383)


In [ ]:
# xy_list = round(gdf[['Longitude', 'Latitude']].astype(float), 5).values.tolist()
# velocity_list = round(gdf['v_Vel'], 2).values.tolist()
# acceleration_list = round(gdf['v_Acc'], 2).values.tolist()

xy_list = round(gdf[['Latitude', 'Longitude']], 5).values.tolist()
velocity_list = round(gdf['v_Vel'], 2).values.tolist()
acceleration_list = round(gdf['v_Acc'], 2).values.tolist()

In [ ]:
velocity_colormap = branca.colormap.StepColormap(colors = ["#edf8e9", "#bae4b3", "#74c476", "#31a354", "#006d2c"],
                                          vmin = gdf["v_Vel"].min(),
                                          vmax = gdf["v_Vel"].max(),
                                          caption = "velocity (feet/second)")


In [ ]:
velocity_colormap

In [ ]:
velocity_colorline = folium.ColorLine(positions = xy_list, colors = velocity_list, colormap = velocity_colormap, weight = 10, opacity = 0.8)
feature_group0.add_child(velocity_colorline)

In [ ]:
acceleration_colormap = branca.colormap.StepColormap(colors = ["#2b8cbe", "#74a9cf", "#ffffbf", "#fdae61", "#d7191c"],
                                          vmin=-12.27,
                                          vmax=12.27,
                                          caption = "acceleration (feet/second square)")

In [ ]:
acceleration_colormap

In [ ]:
acceleration_colorline = folium.ColorLine(positions = xy_list, colors = acceleration_list, colormap = acceleration_colormap, weight = 10, opacity = 0.8)
feature_group1.add_child(acceleration_colorline)

In [ ]:
for lon, lat, velocity, acceleration in zip(gdf["Longitude"], gdf["Latitude"], gdf["v_Vel"], gdf["v_Acc"]):
  folium.Marker(location=[lat, lon], popup = folium.Popup("The longitude and latitude are: " + 
                                                          str(round(float(lon), 5)) + ", " + 
                                                          str(round(float(lat), 5)), max_width = 500),
                                                          icon = folium.Icon(color = "#fee0d2", icon="info-sign")).add_to(marker_cluster)
  folium.Circle(location=[lat, lon], tooltip = "The velocity is " + str(velocity) + " feet/second.", 
                radius = 2, color = "#e0e0e0", fill = True, fill_color = "e0e0e0", weight = 1, opacity = 0.5).add_to(feature_group0)
  folium.Circle(location=[lat, lon], tooltip = "The acceleration is " + str(acceleration) + " feet/second square.", 
                radius = 2, color = "#e0e0e0", fill = True, fill_color = "e0e0e0", weight = 1, opacity = 0.5).add_to(feature_group1)             

In [ ]:
bad = gdf.query("v_Acc < -12.27 or v_Acc > 12.27", inplace = False)
if bad.empty is False:
  for lon, lat, tip in zip(bad["Longitude"], bad["Latitude"], bad["v_Acc"]):
    folium.Circle(location = [lat, lon], tooltip = "The bad acceleration/deceleration is " + str(round(tip, 3)) + " feet per second square", 
                  radius = 5, color = "#de2d26", fill = True, fill_color = "#de2d26", weight = 1).add_to(feature_group3)

In [ ]:
m.add_child(feature_group0)
m.add_child(feature_group1)
m.add_child(feature_group2)
m.add_child(feature_group3)
velocity_colormap.add_to(m)
acceleration_colormap.add_to(m)
m.add_child(folium.map.LayerControl())